In [7]:
%%configure -f
{"executorCores": 4, "executorMemory": "8096M", "conf": {"spark.default.parallelism": 1000}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1572950834699_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1572950834699_0002,pyspark,idle,Link,Link,✔


In [8]:
import pandas as pd

import gensim
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import string
import numpy as np
from langdetect import detect
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType, DoubleType, MapType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
import spacy

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df = spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized")
df = df.distinct()
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- entity_id: string (nullable = true)
 |-- vendor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- id_capiq: integer (nullable = true)
 |-- id_bvd: string (nullable = true)
 |-- primary_bvd_node: string (nullable = true)
 |-- name: string (nullable = true)
 |-- business_description: string (nullable = true)
 |-- primary_sic_node: string (nullable = true)
 |-- primary_naics_node: string (nullable = true)
 |-- primary_nace_node: string (nullable = true)
 |-- has_listed_equity: string (nullable = true)
 |-- country_of_incorporation: string (nullable = true)
 |-- country_of_incorporation_iso: string (nullable = true)
 |-- operating_countries: string (nullable = true)
 |-- number_employees: integer (nullable = true)
 |-- ccc: double (nullable = true)
 |-- revenue: double (nullable = true)
 |-- total_revenue: double (nullable = true)
 |-- net_inc_incl_xo: double (nullable = true)
 |-- total_debt_repaid: double (nullable = true)
 |-- ebitda: double (nullable = true

In [10]:
nlp = spacy.load("en_core_web_sm")

def process_text(text, company_names):
    company_names = company_names.split(", ")
    if not text:
        return ""
    doc = nlp(text)
    result = []
    for sent in doc.sents:
        sent_string = sent.string
        if "was founded in" in sent_string or "a subsidiary of" in sent_string or "based in" in sent_string or "was formerly known as" in sent_string:
            continue
            
        if "[source" in sent_string:
            start_index = sent_string.find('[source')
            end_index = sent_string.find(']', start_index)
            sent_string = sent_string[:start_index] + sent_string[end_index+1:]
            
        for company_name in company_names:
            if company_name in sent_string:
                sent_string = sent_string.replace(company_name, "")
                
        result.append(sent_string)
    return " ".join(result)

process_text_udf = udf(process_text, StringType())
df = df.withColumn("clean_description", process_text_udf('short_description', 'name')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
(df
 .write
 .parquet("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_clean", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# df_new = spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_deduped")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…